In [1]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [2]:
#READ

#read data
df = pd.read_csv("data.csv")

#select data
selected = ["Weapon", "Range", "DropOff", 
            "Zero Damage Range", "RPM","Capacity","ReloadSpeedFromEmpty(ms)", 
            "HeadshotMultiplier", "WeaponBonus", "MaxBonus%", "regularDMG", "minimumDMG"]
df = df[selected]

#CLEAN

#replacement
df["WeaponBonus"] = df["WeaponBonus"].str.replace("-", "No Bonus").astype(str)


df['ReloadSpeedFromEmpty(ms)'] = df['ReloadSpeedFromEmpty(ms)'].str.replace("-", "0").astype(str)


df['MaxBonus%'] = df['MaxBonus%'].str.replace('%', '').astype(str)
df['MaxBonus%'] = df['MaxBonus%'].str.replace('-', '0').astype(str)

df['HeadshotMultiplier'] = df['HeadshotMultiplier'].str.replace('1 \+ Bonus', '1',regex=True).astype(str)
df['HeadshotMultiplier'] = df['HeadshotMultiplier'].str.replace('1\+bonus', '1',regex=True).astype(str) #this also happens
#type setting. (Just in case it needs to be done)

df = df.astype({"Weapon": str, "Range": int, "DropOff": int, 
            "Zero Damage Range": int, "RPM": int,"Capacity": int,"ReloadSpeedFromEmpty(ms)": int, 
            "HeadshotMultiplier": float, "WeaponBonus": str, "MaxBonus%": int, "regularDMG": int, "minimumDMG": int})


In [3]:
weights = {
    "regularDMG": 0.4,
    "RPM": 0.3,
    "ReloadSpeedFromEmpty(ms)": -0.2,
    "HeadshotMultiplier": 0.1,
    "minimumDMG": -0.3,
    "MaxBonus%": 0.3
}